In [39]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import substring
import sys

sys.path.append("routeimager/")

import routeimager.config as cfg
from routeimager.utils import read_maps_api_key, geocode_address, tile_session_token, compute_route
from routeimager.image import RouteTiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
api_key = read_maps_api_key()
route = compute_route(origin="Fregattgatan 3, 21113 Malmö, Sweden", destination="Stockholm, Sweden", api_key=api_key)

In [17]:
line = LineString(route['routes'][0]['polyline']['geoJsonLinestring']['coordinates'])
gdf = gpd.GeoDataFrame(geometry=[line], crs='EPSG:4326')
gdf = gdf.to_crs('EPSG:3006')


In [29]:
route['routes'][0]['polyline']['geoJsonLinestring']['coordinates'][0][1]

55.616713

In [42]:
rt = RouteTiler('test')

tiles = []
for coord in line.coords:
    tiles.append(rt._lat_lng_to_tile_coordinate(*coord))

In [12]:
import math
import numpy as np
from shapely.geometry import LineString

def _lat_lng_to_tile_coordinate(lng, lat):
    lat_rad = math.radians(lat)
    n = 2.0 ** 12
    x = int((lng + 180.0) / 360.0 * n)
    y = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (x, y)

def _tile_coordinate_to_wgs_bounds(x, y):
    n = 2.0 ** 12
    lon_deg_left = x / n * 360.0 - 180.0
    lon_deg_right = (x + 1) / n * 360.0 - 180.0
    lat_rad_top = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    lat_rad_bottom = math.atan(math.sinh(math.pi * (1 - 2 * (y + 1) / n)))
    lat_deg_top = math.degrees(lat_rad_top)
    lat_deg_bottom = math.degrees(lat_rad_bottom)
    #left_bottom_proj = self.project(lon_deg_left, lat_deg_bottom)
    #right_top_proj = self.project(lon_deg_right, lat_deg_top)
    return (lon_deg_left, lat_deg_bottom, lon_deg_right, lat_deg_top)


def wgs84_to_pixel(lon, lat):
    tilesize=1024
    tile = _lat_lng_to_tile_coordinate(lon, lat)
    tile_x_idx = tile[0] - 2192
    tile_y_idx = 1284 - tile[1]
    bbox = _tile_coordinate_to_wgs_bounds(*tile)

    #lat_len, lon_len = meters_per_degree_lat_lon(math.radians((bbox[1] + bbox[3]) / 2))

    # Calculate the physical dimensions of the bounding box
    #lat_diff_meters = (bbox[3] - bbox[1]) * lat_len
    #lon_diff_meters = (bbox[2] - bbox[0]) * lon_len
    
    # Calculate scaling factors
    x_scale = tilesize / (bbox[2] - bbox[0])
    y_scale = tilesize / (bbox[3] - bbox[1])

    # Convert to pixel coordinates
    pixel_x = int((lon - bbox[0]) * x_scale + tile_x_idx*tilesize)
    pixel_y = int((lat - bbox[1]) * y_scale + tile_y_idx*tilesize)
    print(f"TileX: {tile_x_idx} X: {pixel_x} TileY: {tile_y_idx} Y: {pixel_y}")
    return pixel_x, pixel_y



# Bounding box coordinates
bbox = (12.919921875, 55.578344672182055, 13.1078125, 55.65799595426721)

# Start and end points for the line within the bounding box
start_point = (bbox[0], bbox[1])
end_point = (bbox[2], bbox[3])

# Generate 10 points linearly interpolated between the start and end points
num_points = 10
lons = np.linspace(start_point[0], end_point[0], num_points)
lats = np.linspace(start_point[1], end_point[1], num_points)

# Combine lons and lats into a list of coordinates
line_coords = list(zip(lons, lats))

# Create a LineString from the coordinates
line = LineString(line_coords)
pixel_coords = [wgs84_to_pixel(x, y) for x, y in line.coords]

svg_polygon = '<polyline points="{}" style="fill:none;stroke:purple;stroke-width:5" />'.format(
    ' '.join([f'{int(x)},{int(y)}' for x, y in pixel_coords]) # Invert y-axis for pixel coordinates
)
svg_content = f'<svg viewBox="0 0 {64512} {83968}" xmlns="http://www.w3.org/2000/svg" version="1.1">{svg_polygon}</svg>'

with open('buffer_polygon_test.svg', 'w') as file:
    file.write(svg_content)

TileX: 3 X: 3072 TileY: 0 Y: 1023
TileX: 3 X: 3315 TileY: 1 Y: 1206
TileX: 3 X: 3558 TileY: 1 Y: 1389
TileX: 3 X: 3801 TileY: 1 Y: 1571
TileX: 3 X: 4044 TileY: 1 Y: 1754
TileX: 4 X: 4288 TileY: 1 Y: 1936
TileX: 4 X: 4531 TileY: 2 Y: 2119
TileX: 4 X: 4774 TileY: 2 Y: 2301
TileX: 4 X: 5017 TileY: 2 Y: 2484
TileX: 5 X: 5261 TileY: 2 Y: 2667


In [8]:
pixel_coords

[(3072, 83968),
 (3315, 82102),
 (3558, 82285),
 (3801, 82467),
 (4044, 82650),
 (4288, 82832),
 (4531, 80967),
 (4774, 81149),
 (5017, 81332),
 (5261, 81515)]